In [1]:
import pandas as pd
import os
import numpy
pd.set_option('display.max_columns', None)

In [2]:
atac_state = pd.read_csv(r"../output_data/atac_state_full.csv")
atac_state['departure_time']  = pd.to_datetime(atac_state['departure_time'],format= '%H:%M:%S' )

/tmp/ipykernel_41222/1646227078.py:1: DtypeWarning: Columns (3,8) have mixed types. Specify dtype option on import or set low_memory=False.
  atac_state = pd.read_csv(r"../output_data/atac_state_full.csv")


In [3]:
tu_feed  = pd.read_csv('../output_data/tu_feed.csv', parse_dates=['tripUpdate.timestamp'])
tu_feed = tu_feed.rename(columns={'tripUpdate.trip.tripId': 'trip_id'})
tu_feed['tripUpdate.timestamp'] = tu_feed['tripUpdate.timestamp'] + pd.DateOffset(hours=2)
vp_feed  = pd.read_csv('../output_data/vp_feed_large.csv', parse_dates=['vehicle.timestamp'])
vp_feed = vp_feed.rename(columns={'vehicle.trip.tripId': 'trip_id'})
vp_feed['vehicle.timestamp'] = vp_feed['vehicle.timestamp'] + pd.DateOffset(hours=2)
# Remove the year from the datetime values
vp_feed['vehicle.timestamp'] = vp_feed['vehicle.timestamp'].dt.strftime('%H:%M:%S')
vp_feed['vehicle.timestamp']  = pd.to_datetime(vp_feed['vehicle.timestamp'],format= '%H:%M:%S' )

vp_feed = vp_feed.rename(columns = {"vehicle.trip.routeId": "route_id", "vehicle.currentStopSequence": "stop_sequence"})

In [4]:

vpa = atac_state.merge(vp_feed, on = ["trip_id", "stop_sequence", "route_id"]	,how = "inner")
vpa = vpa[["trip_id", "departure_time", "stop_sequence", "shape_dist_traveled", 
           "vehicle.currentStatus", "vehicle.timestamp", "vehicle.position.odometer"]]

In [5]:
atac_state.head(1)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,shape_dist_traveled,timepoint,route_id,service_id,trip_headsign
0,1#1-2,07:30:00,1900-01-01 07:30:00,74761,1,NaN,0,1,T01,1#28,TUSCOLANA/ANAGNINA


In [6]:
vp_feed.head(1)

,id,isDeleted,trip_id,vehicle.trip.startTime,vehicle.trip.startDate,route_id,vehicle.trip.directionId,vehicle.position.latitude,vehicle.position.longitude,stop_sequence,vehicle.currentStatus,vehicle.timestamp,vehicle.stopId,vehicle.vehicle.id,vehicle.vehicle.label,vehicle.position.odometer,vehicle.occupancyStatus,vehicle.trip.scheduleRelationship,max_time
0,1,False,0#725-22,17:03:00,20230906,089,0.0,41.881683,12.361637,1,STOPPED_AT,1900-01-01 16:58:36,75708.0,197,246,NaN,NaN,NaN,2023-09-06 14:59:01


In [7]:
vpa.loc[vpa["vehicle.timestamp"] > vpa["departure_time"]]

,trip_id,departure_time,stop_sequence,shape_dist_traveled,vehicle.currentStatus,vehicle.timestamp,vehicle.position.odometer
0,0#2196-12,1900-01-01 12:06:48,5,1716,IN_TRANSIT_TO,1900-01-01 12:08:06,1729.0
1,0#2196-12,1900-01-01 12:06:48,5,1716,IN_TRANSIT_TO,1900-01-01 12:07:04,1359.0
3,0#2196-12,1900-01-01 12:08:34,6,2342,IN_TRANSIT_TO,1900-01-01 12:09:08,2310.0
4,0#2196-12,1900-01-01 12:09:14,7,2587,IN_TRANSIT_TO,1900-01-01 12:10:40,2563.0
17,0#2196-12,1900-01-01 12:16:14,11,5078,IN_TRANSIT_TO,1900-01-01 12:22:02,4554.0
...,...,...,...,...,...,...,...
416862,0#2273-20,1900-01-01 15:39:00,1,0,STOPPED_AT,1900-01-01 15:43:22,72.0
416863,0#2273-20,1900-01-01 15:39:00,1,0,STOPPED_AT,1900-01-01 15:41:08,5.0
416897,0#2273-24,1900-01-01 18:07:00,1,0,STOPPED_AT,1900-01-01 18:07:10,19.0
416909,0#3379-4,1900-01-01 15:57:00,1,0,STOPPED_AT,1900-01-01 15:57:22,NaN


In [8]:
result = vpa.sort_values(["trip_id", "departure_time", "stop_sequence","vehicle.timestamp"], ascending = [True, True, True, True]).drop_duplicates(subset=["trip_id", "departure_time", "stop_sequence"], keep="first")
result

,trip_id,departure_time,stop_sequence,shape_dist_traveled,vehicle.currentStatus,vehicle.timestamp,vehicle.position.odometer
175045,0#1000-10,1900-01-01 12:15:00,1,0,STOPPED_AT,1900-01-01 12:16:44,10961.0
175029,0#1000-10,1900-01-01 12:18:40,3,529,IN_TRANSIT_TO,1900-01-01 12:32:48,441.0
175030,0#1000-10,1900-01-01 12:20:42,4,819,IN_TRANSIT_TO,1900-01-01 12:33:50,748.0
175031,0#1000-10,1900-01-01 12:24:26,6,1356,IN_TRANSIT_TO,1900-01-01 12:34:54,1366.0
175023,0#1000-10,1900-01-01 12:31:00,7,2295,IN_TRANSIT_TO,1900-01-01 12:35:26,1393.0
...,...,...,...,...,...,...,...
175007,0#999-13,1900-01-01 18:43:26,15,5183,IN_TRANSIT_TO,1900-01-01 18:30:02,NaN
175015,0#999-13,1900-01-01 18:46:04,16,5669,IN_TRANSIT_TO,1900-01-01 18:33:06,NaN
175021,0#999-13,1900-01-01 18:48:50,17,6190,IN_TRANSIT_TO,1900-01-01 18:36:42,NaN
175011,0#999-13,1900-01-01 18:50:38,18,6522,IN_TRANSIT_TO,1900-01-01 18:41:54,NaN


In [9]:
# Define a function to calculate lag within each group based on sorted 'stop_sequence'
def add_lag(group):
    group = group.sort_values(by='stop_sequence', ascending = False)  # Sort within the group
    group['next_stop_sequence'] = group['stop_sequence'].shift(1)
    group['next_vehicle.timestamp'] = group['vehicle.timestamp'].shift(1)
    return group

# Apply the function within each group based on 'trip_id'
result = result.groupby('trip_id').apply(add_lag)

In [10]:
result.reset_index(drop= True, inplace = True)
result["timestamp_differences"] = result['next_vehicle.timestamp'] - result['vehicle.timestamp']

In [11]:
result

,trip_id,departure_time,stop_sequence,shape_dist_traveled,vehicle.currentStatus,vehicle.timestamp,vehicle.position.odometer,next_stop_sequence,next_vehicle.timestamp,timestamp_differences
0,0#1000-10,1900-01-01 12:37:42,11,3803,IN_TRANSIT_TO,1900-01-01 12:43:44,3006.0,NaN,NaT,NaT
1,0#1000-10,1900-01-01 12:34:04,10,2987,IN_TRANSIT_TO,1900-01-01 12:42:42,2961.0,11.0,1900-01-01 12:43:44,0 days 00:01:02
2,0#1000-10,1900-01-01 12:33:18,9,2814,IN_TRANSIT_TO,1900-01-01 12:41:40,2667.0,10.0,1900-01-01 12:42:42,0 days 00:01:02
3,0#1000-10,1900-01-01 12:31:48,8,2478,IN_TRANSIT_TO,1900-01-01 12:40:38,2350.0,9.0,1900-01-01 12:41:40,0 days 00:01:02
4,0#1000-10,1900-01-01 12:31:00,7,2295,IN_TRANSIT_TO,1900-01-01 12:35:26,1393.0,8.0,1900-01-01 12:40:38,0 days 00:05:12
...,...,...,...,...,...,...,...,...,...,...
165876,0#999-13,1900-01-01 18:21:26,6,1356,IN_TRANSIT_TO,1900-01-01 18:09:52,NaN,7.0,1900-01-01 18:10:24,0 days 00:00:32
165877,0#999-13,1900-01-01 18:18:54,5,991,IN_TRANSIT_TO,1900-01-01 18:07:16,NaN,6.0,1900-01-01 18:09:52,0 days 00:02:36
165878,0#999-13,1900-01-01 18:17:42,4,819,IN_TRANSIT_TO,1900-01-01 18:06:44,NaN,5.0,1900-01-01 18:07:16,0 days 00:00:32
165879,0#999-13,1900-01-01 18:15:40,3,529,IN_TRANSIT_TO,1900-01-01 18:04:10,NaN,4.0,1900-01-01 18:06:44,0 days 00:02:34


In [12]:
# Define a function to apply your condition
def set_new_column(row):
    if row['next_stop_sequence'] - row['stop_sequence'] == 1:
        return row['vehicle.timestamp'] + row["timestamp_differences"]/2
    else:
        return row['vehicle.timestamp']  # Set new_column to be the value from column2

# Apply the function to create the new column
result['estimated_departure_time'] = result.apply(set_new_column, axis=1)

In [13]:
result

,trip_id,departure_time,stop_sequence,shape_dist_traveled,vehicle.currentStatus,vehicle.timestamp,vehicle.position.odometer,next_stop_sequence,next_vehicle.timestamp,timestamp_differences,estimated_departure_time
0,0#1000-10,1900-01-01 12:37:42,11,3803,IN_TRANSIT_TO,1900-01-01 12:43:44,3006.0,NaN,NaT,NaT,1900-01-01 12:43:44
1,0#1000-10,1900-01-01 12:34:04,10,2987,IN_TRANSIT_TO,1900-01-01 12:42:42,2961.0,11.0,1900-01-01 12:43:44,0 days 00:01:02,1900-01-01 12:43:13
2,0#1000-10,1900-01-01 12:33:18,9,2814,IN_TRANSIT_TO,1900-01-01 12:41:40,2667.0,10.0,1900-01-01 12:42:42,0 days 00:01:02,1900-01-01 12:42:11
3,0#1000-10,1900-01-01 12:31:48,8,2478,IN_TRANSIT_TO,1900-01-01 12:40:38,2350.0,9.0,1900-01-01 12:41:40,0 days 00:01:02,1900-01-01 12:41:09
4,0#1000-10,1900-01-01 12:31:00,7,2295,IN_TRANSIT_TO,1900-01-01 12:35:26,1393.0,8.0,1900-01-01 12:40:38,0 days 00:05:12,1900-01-01 12:38:02
...,...,...,...,...,...,...,...,...,...,...,...
165876,0#999-13,1900-01-01 18:21:26,6,1356,IN_TRANSIT_TO,1900-01-01 18:09:52,NaN,7.0,1900-01-01 18:10:24,0 days 00:00:32,1900-01-01 18:10:08
165877,0#999-13,1900-01-01 18:18:54,5,991,IN_TRANSIT_TO,1900-01-01 18:07:16,NaN,6.0,1900-01-01 18:09:52,0 days 00:02:36,1900-01-01 18:08:34
165878,0#999-13,1900-01-01 18:17:42,4,819,IN_TRANSIT_TO,1900-01-01 18:06:44,NaN,5.0,1900-01-01 18:07:16,0 days 00:00:32,1900-01-01 18:07:00
165879,0#999-13,1900-01-01 18:15:40,3,529,IN_TRANSIT_TO,1900-01-01 18:04:10,NaN,4.0,1900-01-01 18:06:44,0 days 00:02:34,1900-01-01 18:05:27


In [15]:
result["estimated_delay"] = (abs(result['estimated_departure_time'] - result['departure_time'])).dt.total_seconds().astype(int)

In [16]:
result

,trip_id,departure_time,stop_sequence,shape_dist_traveled,vehicle.currentStatus,vehicle.timestamp,vehicle.position.odometer,next_stop_sequence,next_vehicle.timestamp,timestamp_differences,estimated_departure_time,estimated_delay
0,0#1000-10,1900-01-01 12:37:42,11,3803,IN_TRANSIT_TO,1900-01-01 12:43:44,3006.0,NaN,NaT,NaT,1900-01-01 12:43:44,362
1,0#1000-10,1900-01-01 12:34:04,10,2987,IN_TRANSIT_TO,1900-01-01 12:42:42,2961.0,11.0,1900-01-01 12:43:44,0 days 00:01:02,1900-01-01 12:43:13,549
2,0#1000-10,1900-01-01 12:33:18,9,2814,IN_TRANSIT_TO,1900-01-01 12:41:40,2667.0,10.0,1900-01-01 12:42:42,0 days 00:01:02,1900-01-01 12:42:11,533
3,0#1000-10,1900-01-01 12:31:48,8,2478,IN_TRANSIT_TO,1900-01-01 12:40:38,2350.0,9.0,1900-01-01 12:41:40,0 days 00:01:02,1900-01-01 12:41:09,561
4,0#1000-10,1900-01-01 12:31:00,7,2295,IN_TRANSIT_TO,1900-01-01 12:35:26,1393.0,8.0,1900-01-01 12:40:38,0 days 00:05:12,1900-01-01 12:38:02,422
...,...,...,...,...,...,...,...,...,...,...,...,...
165876,0#999-13,1900-01-01 18:21:26,6,1356,IN_TRANSIT_TO,1900-01-01 18:09:52,NaN,7.0,1900-01-01 18:10:24,0 days 00:00:32,1900-01-01 18:10:08,678
165877,0#999-13,1900-01-01 18:18:54,5,991,IN_TRANSIT_TO,1900-01-01 18:07:16,NaN,6.0,1900-01-01 18:09:52,0 days 00:02:36,1900-01-01 18:08:34,620
165878,0#999-13,1900-01-01 18:17:42,4,819,IN_TRANSIT_TO,1900-01-01 18:06:44,NaN,5.0,1900-01-01 18:07:16,0 days 00:00:32,1900-01-01 18:07:00,642
165879,0#999-13,1900-01-01 18:15:40,3,529,IN_TRANSIT_TO,1900-01-01 18:04:10,NaN,4.0,1900-01-01 18:06:44,0 days 00:02:34,1900-01-01 18:05:27,613


In [17]:
result["estimated_delay"].mean()

506.0400588373593